## Purpose

- test sampling 
----

## TODO

1. save sparse matrices for import 

In [36]:
%load_ext memory_profiler
%matplotlib inline

import os, sys, glob, re, math, pickle
import phate,scprep,magic,meld,umap
import graphtools as gt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import cmocean,time,random
# import scvelo as scv
# import community, phenograph
from sklearn import metrics
from sklearn import model_selection
from scipy import sparse
import scanpy as sc
from sklearn.dummy import DummyClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch_geometric.data import Data
from torch_geometric.data import NeighborSampler


# default for `matplotlib` font size
plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
sc.set_figure_params(dpi=300,dpi_save=600,
                     frameon=False,
                     fontsize=9)
plt.rcParams['savefig.dpi'] = 600
sc.settings.verbosity=3

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [2]:
if False :
    pfp = '/home/ngr4/project/scnd/results/'
    pdfp = '/home/ngr4/project/scnd/data/processed/'
    sc.settings.figdir=pfp

    # load 'test' data for val
    with open(os.path.join(pdfp,'earlyDev_test.pickle'),'rb') as f:
        data_test = pickle.load(f) # test if b, np.sum(b['adj']==adj_train)==adj_train.shape[0]**2
    labels_val=data_test['labels']
    labels_val=torch.LongTensor(labels_val)
    adj_val = data_test['adj']
    adj_val = torch.from_numpy(adj_val)
    features_val = data_test['node_features']
    features_val = torch.from_numpy(features_val)
    
if True :
    pdfp='/home/ngr4/project/scgraph/data/processed/'
    %memit adata = sc.read(os.path.join(pdfp,'induction_train_50p.h5ad'))

In [3]:
def scipysparse2torchsparse(x) :
    '''
    Input: scipy csr_matrix
    Returns: torch tensor in experimental sparse format

    REF: Code adatped from [PyTorch discussion forum](https://discuss.pytorch.org/t/better-way-to-forward-sparse-matrix/21915>)
    '''
    samples=x.shape[0]
    features=x.shape[1]
    values=x.data
    coo_data=x.tocoo()
    indices=torch.LongTensor([coo_data.row,coo_data.col]) # OR transpose list of index tuples
    t=torch.sparse.FloatTensor(indices,torch.from_numpy(values).float(),[samples,features])
    return indices,t

In [33]:
labels = adata.obs['MS']
labels = torch.LongTensor(labels)
edge_index,edge_feat = scipysparse2torchsparse(adata.uns['neighbors']['connectivities'])
%memit X_idx,X = scipysparse2torchsparse(adata.X)

In [35]:
data = Data(x=X, edge_index=edge_index, y=labels)

In [40]:
loader = NeighborSampler(data,size=10,add_self_loops=True,
                         num_hops=1,batch_size=512,shuffle=True)

In [43]:
batch = loader.__produce_subgraph__(loader.__get_batches__()[0])

In [49]:
batch

Data(b_id=[512], e_id=[4996], edge_index=[2, 4996], n_id=[5045], sub_b_id=[512])

In [48]:
batch.num_nodes

5045

In [58]:
data.x

tensor(indices=tensor([[    0,     0,     0,  ..., 43865, 43865, 43865],
                       [    8,    13,    21,  ..., 21980, 21982, 21993]]),
       values=tensor([0.9790, 0.9790, 0.9790,  ..., 1.2838, 0.9078, 0.9078]),
       size=(43866, 22005), nnz=59179416, layout=torch.sparse_coo)

In [56]:
data.x[batch.b_id,:]

RuntimeError: sparse tensors do not have strides

In [63]:
x = torch.rand([3,4])
x[[0,1,2],[0,2,2]]=0
x

tensor([[0.0000, 0.3598, 0.0772, 0.5382],
        [0.5501, 0.1705, 0.0000, 0.7037],
        [0.9368, 0.1809, 0.0000, 0.8275]])

In [64]:
x.nonzero()

tensor([[0, 1],
        [0, 2],
        [0, 3],
        [1, 0],
        [1, 1],
        [1, 3],
        [2, 0],
        [2, 1],
        [2, 3]])